<a href="https://colab.research.google.com/github/kumarmohit0911/AAA/blob/main/Basic_gans_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import dataset, DataLoader

In [2]:
class Descriminator(nn.Module):
  def __init__(self,img_dim):
    super().__init__()
    self.disc = nn.Sequential(
        nn.Linear(img_dim,128),
        nn.LeakyReLU(0.1),
        nn.Linear(128,1),
        nn.Sigmoid()
    )

  def forward(self,x):
    x = self.disc(x)
    return x

In [3]:
class Generator(nn.Module):
  def __init__(self,z_dim,img_dim):
    super().__init__()
    self.gen = nn.Sequential(
        nn.Linear(z_dim,256),
        nn.LeakyReLU(0.1),
        nn.Linear(256,img_dim),
        nn.Tanh()
    )
  def forward(self,x):
    x = self.gen(x)
    return x

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)
lr = 3e-4
z_dim = 64
img_dim = 28*28*1
batch_size = 32
epochs = 50

cuda


In [5]:
disc = Descriminator(img_dim).to(device)
gen = Generator(z_dim,img_dim).to(device)
fixed_noise = torch.randn((batch_size,z_dim)).to(device)

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,),(0.5,))]
)

In [17]:
dataset = datasets.MNIST(root = "dataset/",transform=transform,download= True)

In [18]:
loader = DataLoader(dataset,batch_size=batch_size,shuffle= True)

In [19]:
opt_disc = optim.Adam(disc.parameters(), lr = lr)
opt_gen = optim.Adam(gen.parameters(), lr = lr )

In [20]:
Criterion = nn.BCELoss()

In [21]:
from torch.utils.tensorboard import SummaryWriter
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")

In [22]:
step = 0

In [ ]:
for epoch in range(epochs):
  for batch_idx,(real,_) in enumerate(loader):
    real = real.view(-1,784).to(device)
    batch_size = real.shape[0]
    #loss fnxn for discriminator
    # max log(D(G)) + log(1-D(G(Z))) --> Z is the noise
    noise = torch.randn(batch_size,z_dim).to(device)
    fake = gen(noise) #fake image generated
    disc_real = disc(real).view(-1) # real image is fed to the descriminator
    lossD_real = Criterion(disc_real,torch.ones_like(disc_real))
    disc_fake = disc(fake).view(-1)
    lossD_fake = Criterion(disc_fake,torch.zeros_like(disc_fake))
    lossD = (lossD_fake + lossD_real )/2
    disc.zero_grad()
    lossD.backward(retain_graph=True)
    opt_disc.step()
    # training generator
    # max(log(D(G(Z)))
    output = disc(fake).view(-1)
    lossG = Criterion(output,torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()